# Sentiment Analysis for Amazon reviews on cellphones and accessories
# Cellphone reviews dataset is at http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz



In [2]:
### Import necessary depencencies

import gzip
import json
import multiprocessing
import sqlite3
from multiprocessing import Process

import pandas as pd
from nltk.tokenize.toktok import ToktokTokenizer

import utils

# ============================================
#   Overall plan to pre-processing dataset 
# ============================================
### 0. Load data
### 1. Prune for local development
### 2. Reformat dates and times for visualization
### 3. Cleaning review text
     a. strip HTML
     b. Removing accented characters
     c. Expanding Contractions
     d. Removing Special Characters
     e. Lemmatizing text
     f. Removing Stopwords
     g. Remove special characters and repeating characters
     g. Spelling corrections





In [3]:
def child_process(number):
    number.value = 0
    while True:
        number.value += 1
        #print(number)

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)


In [4]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


# ============================================
# 0.  Load data 
# ============================================


In [5]:
dfraw = getDF('./data/cell_reviews_5core.json.gz')

print('Total Rows: ' + str(len(dfraw)))
# Sample of raw dataset
dfraw.head(20)



Total Rows: 1128437


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,3.0,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,2.0,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4.0,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN
5,2.0,True,"01 27, 2014",A1DW2L6XCC5TJS,7508492919,NaN,Amazon Customer,The product looked exactly like the picture an...,Not so happy,1390780800,NaN,NaN
6,3.0,True,"01 23, 2014",AQC61R4UST7UH,7508492919,NaN,DaMara Estep,I FINALLY got my case today. It took forever t...,It's cute!,1390435200,NaN,NaN
7,5.0,True,"01 17, 2014",A31OVFL91BCKXG,7508492919,NaN,Ashley Nicole Miller,It is a very cute case. None of the jewels hav...,Cute case,1389916800,NaN,NaN
8,1.0,True,"12 27, 2013",A1K0VLK6O5Z22M,7508492919,NaN,BeeLove21,DO NOT BUY! this item is seriously cheap as he...,WORST ITEM!,1388102400,NaN,NaN
9,4.0,True,"12 16, 2013",A1K3BWU73YB44P,7508492919,NaN,Mrs. Ochoa,I really love this case... you have to keep yo...,Pretty Cute!,1387152000,NaN,NaN


# ============================================
# 1. Prune for local development
# ============================================

In [6]:
trial=0

#subset for local runs, will remove on final runs or on server
five=(dfraw['overall'] == 5.0)
four=(dfraw['overall'] >= 4.0) & (dfraw['overall'] < 5.0)
three=(dfraw['overall'] >= 3.0) & (dfraw['overall'] < 4.0)
two=(dfraw['overall'] >= 2.0) & (dfraw['overall'] < 3.0)
one=(dfraw['overall'] >= 1.0) & (dfraw['overall'] < 2.0)
zero=(dfraw['overall'] >= 0.0) & (dfraw['overall'] < 1.0)

df=pd.DataFrame(columns = dfraw.columns)
if(trial>0):
    df=dfraw[five].iloc[0:trial]
    df=df.append(dfraw[four].iloc[0:trial])
    #ignoring star rating=3 as it is neither positive or negative
    #df=df.append(dfraw[three].iloc[0:trial])
    df=df.append(dfraw[two].iloc[0:trial])
    df=df.append(dfraw[one].iloc[0:trial])
    df=df.append(dfraw[zero].iloc[0:trial])
else:
    #ignoring star rating=3 as it is neither positive or negative
    df=dfraw[five]
    df=df.append(dfraw[four])
    df=df.append(dfraw[two])
    df=df.append(dfraw[one])
    df=df.append(dfraw[zero])
    
print('Total Rows: ' + str(len(df)))
# Sample of Unprocessed reviews pruned for local development
df.head(10)


Total Rows: 2000


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
7,5.0,True,"01 17, 2014",A31OVFL91BCKXG,7508492919,NaN,Ashley Nicole Miller,It is a very cute case. None of the jewels hav...,Cute case,1389916800,NaN,NaN
11,5.0,True,"10 23, 2013",A2ZB7KGUSBR9P3,7508492919,NaN,E. Bryce,Another great product that my daughter she use...,Bling bling for iPhone 4S,1382486400,NaN,NaN
14,5.0,True,"09 10, 2013",A18U23JWTMQX5C,7508492919,NaN,KaitlynxO625,Beautiful quality and outstanding product! Eve...,I can't stop using this case!,1378771200,NaN,NaN
15,5.0,True,"08 28, 2013",A1JQUCTFM4UKMQ,7508492919,NaN,M. Antillon,It is such a good case for a low price. I have...,I love it,1377648000,NaN,NaN
18,5.0,True,"06 4, 2013",A29KSIE8BKYVQN,7508492919,NaN,Janine B.,Super durable and I get compliments on it dail...,Good case,1370304000,NaN,NaN
19,5.0,True,"06 3, 2013",A2CQO0FORCTC2R,7508492919,NaN,Sulli,I have used this case for a couple weeks & so ...,very sparkly,1370217600,NaN,NaN
21,5.0,True,"05 4, 2013",A2ROMLP8COJ6JA,7508492919,NaN,Mary Beth Anderson,I chose this case because it is so beautiful. ...,Beautiful,1367625600,NaN,NaN
22,5.0,True,"04 24, 2013",A23MRCVKI8M8OY,7508492919,NaN,T...,Of all my cases this is the one that I have on...,pink bow,1366761600,NaN,NaN


# =================================================
# 2. Reformat dates and convert rating to sentiment
# =================================================

In [7]:
#infer sentiment positive and negative based on star reviews and add to dataset
#negative= star reviews 0,1,2,
#positive= star reviews 4,5
#create new column for sentiment
#sentiment column is labeled target
df['sentiment']=1
df.loc[(df['overall'] >= 0.0) & (df['overall'] < 3.0), 'sentiment'] = 0

#convert review date to standard format
df['Review_Time']= pd.to_datetime(df['reviewTime'])
df['Month']=df['Review_Time'].dt.month
df['Year']=df['Review_Time'].dt.year


#drop uneeded columns
df.drop('reviewTime', axis=1,inplace=True)
df.drop('style', axis=1,inplace=True)
df.drop('image', axis=1,inplace=True)

print('Total Rows: ' + str(len(df)))
# Sample of raw dataset with dates reformatted. Notice new month and year columns
df.head(20)


Total Rows: 2000


,overall,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,Review_Time,Month,Year
0,5.0,True,A24E3SXTC62LJI,7508492919,Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,2014-08-04,8,2014
1,5.0,True,A269FLZCB4GIPV,7508492919,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,2014-02-12,2,2014
7,5.0,True,A31OVFL91BCKXG,7508492919,Ashley Nicole Miller,It is a very cute case. None of the jewels hav...,Cute case,1389916800,NaN,2014-01-17,1,2014
11,5.0,True,A2ZB7KGUSBR9P3,7508492919,E. Bryce,Another great product that my daughter she use...,Bling bling for iPhone 4S,1382486400,NaN,2013-10-23,10,2013
14,5.0,True,A18U23JWTMQX5C,7508492919,KaitlynxO625,Beautiful quality and outstanding product! Eve...,I can't stop using this case!,1378771200,NaN,2013-09-10,9,2013
15,5.0,True,A1JQUCTFM4UKMQ,7508492919,M. Antillon,It is such a good case for a low price. I have...,I love it,1377648000,NaN,2013-08-28,8,2013
18,5.0,True,A29KSIE8BKYVQN,7508492919,Janine B.,Super durable and I get compliments on it dail...,Good case,1370304000,NaN,2013-06-04,6,2013
19,5.0,True,A2CQO0FORCTC2R,7508492919,Sulli,I have used this case for a couple weeks & so ...,very sparkly,1370217600,NaN,2013-06-03,6,2013
21,5.0,True,A2ROMLP8COJ6JA,7508492919,Mary Beth Anderson,I chose this case because it is so beautiful. ...,Beautiful,1367625600,NaN,2013-05-04,5,2013
22,5.0,True,A23MRCVKI8M8OY,7508492919,T...,Of all my cases this is the one that I have on...,pink bow,1366761600,NaN,2013-04-24,4,2013


# ============================================
#   3. Clean data as below 
# ============================================
##     a.  strip HTML: using regex, not BeautifulSoup (which is slow)
##     b.  Removing accented characters using Regex
##     c.  Expanding Contractions using a map of common contractions
##     d.  Removing Special Characters using Regex
##     e.  Lemmatizing text using Spacy and WordNet
##     f.  Removing Stopwords using ToktokTokenizer (NLTK)  
##     g.  Removing special characters and repeating characters using Regex
##     h.  Spelling corrections using TextBlob


In [ ]:
#drop table because we will reload the table in this run
con = sqlite3.connect('./data/amazon_cellreviews.db')
dropQuery = "DROP TABLE IF EXISTS reviews";
con.execute(dropQuery)


In [ ]:
def writeToDatabase(dfdb):
    dfdb.to_sql('reviews', con, if_exists="append")
    con.commit()
        

In [12]:
#nltk.download('all', halt_on_error = True)

tokenizer = ToktokTokenizer()
#number of processes to run in multiprocessing
processors=16
#number of observations
size=len(df)
#number of observations to normalize in each process
iterSize=round(size/processors)
#holds the processes
processes=[]

print('To process: ' + str(size))
 
i=0

#dfholder=[]
parent_conn_holder=[]

for i in range(0,processors):
    start=i*iterSize
    stop=start+iterSize
    if(stop>size):
        stop=size
    #split df for parallel proc
    dflist=df.iloc[start:stop]
    # creating a pipe 
    parent_conn, child_conn = multiprocessing.Pipe() 
    p = Process(target=utils.multiprocNormalize, args=(dflist,child_conn,"proc"+str(i)))
    #dfholder.append(utils.multiprocNormalize(dflist,"proc"+str(i)))
    processes.append(p)
    p.start()
    parent_conn_holder.append(parent_conn)
    #dflocal.head(10)

for parent_conn in parent_conn_holder:
    dfdb=parent_conn.recv()
    writeToDatabase(dfdb)
    del dfdb


for p in processes:
    p.join()


   

To process: 2000


ValueError: Length of values does not match length of index

In [ ]:
print('done')

In [11]:
con1 = sqlite3.connect('./data/amazon_cellreviews.db')
#check database
sql = " SELECT * FROM reviews where overall=2.0"
dfdb = pd.read_sql_query(sql, con1)
con1.close()

,overall,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,Clean_Review,vote,Review_Time,Month,Year
0,5.0,True,A24E3SXTC62LJI,7508492919,Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,look even well person careful drop phone often...,NaN,2014-08-04,8,2014
1,5.0,True,A269FLZCB4GIPV,7508492919,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,want spend whole lot cash want great deal shop...,NaN,2014-02-12,2,2014
7,5.0,True,A31OVFL91BCKXG,7508492919,Ashley Nicole Miller,It is a very cute case. None of the jewels hav...,Cute case,1389916800,it cute case none jewel fall bow glue well com...,NaN,2014-01-17,1,2014
11,5.0,True,A2ZB7KGUSBR9P3,7508492919,E. Bryce,Another great product that my daughter she use...,Bling bling for iPhone 4S,1382486400,another great product daughter use long time i...,NaN,2013-10-23,10,2013
14,5.0,True,A18U23JWTMQX5C,7508492919,KaitlynxO625,Beautiful quality and outstanding product! Eve...,I can't stop using this case!,1378771200,beautiful quality outstanding product everyone...,NaN,2013-09-10,9,2013
15,5.0,True,A1JQUCTFM4UKMQ,7508492919,M. Antillon,It is such a good case for a low price. I have...,I love it,1377648000,it good case low price i right never problem b...,NaN,2013-08-28,8,2013
18,5.0,True,A29KSIE8BKYVQN,7508492919,Janine B.,Super durable and I get compliments on it dail...,Good case,1370304000,super durable i get compliment daily i find on...,NaN,2013-06-04,6,2013
19,5.0,True,A2CQO0FORCTC2R,7508492919,Sulli,I have used this case for a couple weeks & so ...,very sparkly,1370217600,i use case couple week far great little bling ...,NaN,2013-06-03,6,2013
21,5.0,True,A2ROMLP8COJ6JA,7508492919,Mary Beth Anderson,I chose this case because it is so beautiful. ...,Beautiful,1367625600,i choose case beautiful everyone like ask coul...,NaN,2013-05-04,5,2013
22,5.0,True,A23MRCVKI8M8OY,7508492919,T...,Of all my cases this is the one that I have on...,pink bow,1366761600,case one i phone right i love love love fit gr...,NaN,2013-04-24,4,2013


In [ ]:
print('Total Rows: ' + str(len(dfdb)))
print('Sample of processed dataset. Notice the column named Clean_Review');
dfdb.head(20)